In [11]:
import pandas as pd
import numpy as np

scale = 0.25

# Domain
x = 200*scale
z = 100*scale
y_air = 30*scale
y_moon = 50*scale
y = y_moon + y_air

# Frequency and Wavelength
f_max = 2*50e6 # Maximum Relevant Frequency for the Ricker Wavelet
c = 3e8

# Permittivity and Density
density = 2 # g/cm^3
eps_bg = 1.919**density # Background Permittivity for Moon
vel_bg = c / eps_bg**0.5

lambda_0 = c / (f_max * eps_bg**0.5)

# # Fresnel Zone - Radius
# R = 0.5 * (lambda_0 * d)**0.5
# y = 4 * R

# Grid Descretization
dx = np.round(lambda_0 / 12, 3)
dy = np.round(lambda_0 / 12, 3)
dz = np.round(lambda_0 / 12, 3)

nx = int(x / dx)
ny = int(y / dy)
nz = int(z / dz)
print(f"nx: {nx}, ny: {ny}, nz: {nz}")

# x_grid, y_grid, z_grid = np.meshgrid(x_vals, y_vals, z_vals, indexing='xy')

# Time Window
time_window = 2 * y_moon / vel_bg

# Print
print(f"Domain: {x} x {y} x {z}")
print(f"Base Permittivity of Moon: {eps_bg}")
print(f"Frequency: {f_max/2}")
print(f"Wavelength: {lambda_0}")
# print(f"Fresnel Zone Radius: {R}")
print(f"Grid: {dx} x {dy} x {dz}")
print(f"Time Window: {time_window}")


nx: 384, ny: 153, nz: 192
Domain: 50.0 x 20.0 x 25.0
Base Permittivity of Moon: 3.682561
Frequency: 50000000.0
Wavelength: 1.5633142261594581
Grid: 0.13 x 0.13 x 0.13
Time Window: 1.5991666666666665e-07


In [12]:
# Editing the Lava Tube Points to Comply with the Domain

data_all = pd.read_csv('../LavaTubeData/LiDAR_InclineCave_TUBE_TLS_points.txt', sep=' ', header=None)
# Convert to numpy arrays X, Y, Z
data_points = data_all.to_numpy()
X = (data_points[:, 0])*scale
Y = (data_points[:, 2])*scale
Z = (data_points[:, 1])*scale

# Find the minimum and maximum values of X, Y, Z, and find their index

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

# Shift all points in X, Y, Z to the center of the domain
X = X - X_min
Y = Y - Y_min
Z = Z - Z_min

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

X = X - (X_min + X_max)/2 + x/2
Y = Y - Y_max + y_moon
Z = Z - (Z_min + Z_max)/2 + z/2

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

print(f"X_min: {X_min}")
print(f"X_max: {X_max}")
print(f"Y_min: {Y_min}")
print(f"Y_max: {Y_max}")
print(f"Z_min: {Z_min}")
print(f"Z_max: {Z_max}")

data_all_shifted = pd.DataFrame(np.column_stack((X, Y, Z)))
data_all_shifted = data_all_shifted.to_numpy()

# Save the new points to vtk
from pyevtk.hl import pointsToVTK
pointsToVTK("LiDAR_InclineCave_TUBE_adjusted", X, Y, Z)

X_min: 8.756347625007038
X_max: 41.24365237499296
Y_min: 6.13000500000004
Y_max: 12.5
Z_min: 4.682617124984972
Z_max: 20.317382875015028


'f:\\Projects\\LunarLeaper\\LLGPR\\src\\LiDAR_InclineCave_TUBE_adjusted.vtu'

In [13]:
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator

permittivity_values = 7.6* np.ones_like(X)

In [15]:
#Grid
# x_val = np.linspace(0, x, 5)
# z_val = np.linspace(0, z, 5)
# y_val = np.linspace(0, y_moon, 5)
x_val = np.linspace(0, x, int(nx*scale))
z_val = np.linspace(0, z, int(ny*scale))
y_val = np.linspace(0, y_moon, int(nz*scale))

In [16]:
meshgrid_x, meshgrid_y, meshgrid_z = np.meshgrid(x_val, y_val, z_val, indexing='ij')
points = np.vstack((meshgrid_x.flatten(), meshgrid_y.flatten(), meshgrid_z.flatten())).T

# Create Delaunay triangulation
tri = Delaunay(np.column_stack((X, Y, Z)))

In [25]:
pointsToVTK("Box", meshgrid_x.flatten(), meshgrid_y.flatten(), meshgrid_z.flatten())

'f:\\Projects\\LunarLeaper\\LLGPR\\src\\Box.vtu'

In [17]:
# Check which points are within the convex hull
# Check for interpolation
simplex_indices = tri.find_simplex(points)
mask = simplex_indices >= 0

In [31]:
# x, y, z for just the mask
x_mask = meshgrid_x.flatten()[mask]
y_mask = meshgrid_y.flatten()[mask]
z_mask = meshgrid_z.flatten()[mask]
points_cave_edge = np.vstack((x_mask, y_mask, z_mask)).T
print(f"Number of Points in Convex Hull: {len(z_mask)}")
print(f"Number of Points in Total: {len(points_cave_edge)}")

Number of Points in Convex Hull: 5282
Number of Points in Total: 5282


In [33]:
from scipy.spatial import ConvexHull

# Calculate the centroid of the original points
centroid_cave = np.mean(points_cave_edge, axis=0)

# Function to shrink each point towards the centroid
def shrink_point(point, centroid, dx=2*dx, dy=2*dy, dz=2*dz):
    direction = point - centroid
    factor = np.array([dx, dy, dz]) / np.linalg.norm(direction)
    shrink_vector = direction * factor
    return point - shrink_vector

# Apply shrinking to all points
points_cave_vacuum_all = np.array([shrink_point(p, centroid_cave, dx, dy, dz) for p in points_cave_edge])

# Compute the new convex hull from shrunken points
convex_cave_vacuum = ConvexHull(points_cave_vacuum_all)

# Extract the new set of coordinates defining the new convex hull
points_cave_vacuum = points_cave_vacuum_all[convex_cave_vacuum.vertices]
print(f"Number of Points in Vacuum Convex Hull: {len(points_cave_vacuum)}")

Number of Points in Vacuum Convex Hull: 224


In [38]:
# pointsToVTK("Vacuum", points_cave_vacuum[:, 0], points_cave_vacuum[:, 1], points_cave_vacuum[:, 2])